In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import urllib.request

# All project files live here in your Google Drive
PROJECT_DIR = "/content/drive/MyDrive/phys2600_cosmic_project"
os.makedirs(PROJECT_DIR, exist_ok=True)

%cd $PROJECT_DIR

remote_dir = "https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/final_projects/proj1_cosmic/"
filenames = [
    "cosmic_API_tests.ipynb",
    "cosmic_assign.ipynb",
    "cosmic_presentation.ipynb",
    "cosmic.py",
]

for filename in filenames:
    remote_url = remote_dir + filename
    local_path = os.path.join(PROJECT_DIR, filename)
    if not os.path.exists(local_path):
        print(f"Copying {filename} to {local_path}")
        urllib.request.urlretrieve(remote_url, local_path)
    else:
        print(f"{filename} already exists at {local_path}")


- Run the first code cell.
- Open `cosmic.py` from the folder `drive/MyDrive/phys2600_cosmic_project` on the left.
- Edit `cosmic.py`, save it, then come back to this notebook and run the cells that `import cosmic`.

# PHYS 2600 - Final Project 1: `cosmic`

## Monte Carlo Simulation of Cosmic Ray Showers 


This is one possible _final project_; __you should choose one (and only one) project to complete and hand in.__  (If you try to work on two, I will grade the more complete one.  But these are big projects so I very strongly do NOT recommend attempting both!)

The deadline for this project is __7:00 PM (Boulder time) on Thursday, December 11.__ Since final grades must be submitted to the university shortly after the end of the exam period, to allow for timely grading _no late submissions will be accepted._

You are permitted to discuss the final projects with other students in the class,  including both the physics and computing aspects, but __your submitted project must be entirely your own__; no direct copying of Python code is permitted, and you must write up the presentation in your own words.

Your completed project will consist of __two (2) files__:


* `cosmic.py`: a Python module (see lecture 25) which implements the functions listed in the Application Progamming Interface (API) below.
* `cosmic_presentation.ipynb`: a Jupyter notebook which uses the code from your Python module to answer the physics questions.  Use of Markdown text, MathJax equations, and plots are all encouraged to make your presentation clear and compelling!

The rubric for grading will be split 50/50 between the code (in your `.py` module and Jupyter notebook) and the response to the physics questions (provided in `cosmic_presentation.ipynb`):
* Basic functionality of code (passes provided tests, follows API specification): __30%__
* __Six (6)__ additional API tests _you write_ (tests are provided and are correct and useful): __10%__
* Documentation of code (docstrings and comments make the code clear): __10%__
* Correct and detailed answers to physics questions: __40%__
* Quality of presentation (clear and readable, appropriate use of Markdown, equations, and plots): __10%__

A _bonus of up to 10%_ is available for the extra-credit "challenge" physics question at the end.  (These challenge questions are meant to be really hard!  But partial credit will be awarded if you make some progress on them.)

# Overview

Every second, the Earth is bombarded by __cosmic rays__: high-energy particles flying at us from the cosmos.  These naturally-accelerated particles have been a treasure trove for research, leading to discoveries of new elementary particles such as the positron and the muon in the early 20th century.  The collisions of a cosmic proton with Earth's atmosphere lead to a spectacular multi-pronged trail of new particles called a __cosmic ray shower__.

For this project, you will build a Monte Carlo simulation (in two dimensions to make things simpler) that will reproduce the cosmic-ray air shower effect.  The set of interactions and particles you study will be limited to five: the proton, electron, pion, muon, and neutrino.  You'll need to account for special relativity, which is a crucial ingredient since the cosmic ray particles are moving very fast!

In addition to making nice simulated pictures of a spectacular physical phenomenon, you'll be able to directly investigate how the flux of cosmic rays in the upper atmosphere translates into detection of muons in ground-based observatories.

# Physics background and important equations

For this project, we will adopt a _simplified model_ of particle physics, which omits a lot of complicated details but includes enough physics to get the right qualitative behavior of cosmic ray showers.

We will consider _only_ the following set of particles, and we will ignore their electric charges completely:

| name | symbol | mass |
|------|---------|-----------|
| proton | $p$ | 938.3 MeV/$c^2$ |
| electron | $e$ | 0.5110 MeV/$c^2$ |
| pion | $\pi$ | 139.6 MeV/$c^2$ |
| muon | $\mu$ | 105.7 MeV/$c^2$ |
| neutrino | $\nu$ | $\sim$ 0 |

("MeV" stands for "mega-electron-volt", a unit of energy; electron-volt based units are common in particle physics.)  The proton and electron are familiar; the muon is a heavier (unstable) cousin of the electron, with the same charge.  Neutrinos are ghostly particles with nearly zero mass and very weak interactions.  Finally, the pion is a bound state of the strong nuclear force, like the proton.

For the kinematics of the proton's interaction, you will also need the mass of the nitrogen-14 nucleus, which is approximately $M_{N14} \approx 13040 \ \textrm{MeV}/c^2$ in the same units.

We're missing some details from the [real Standard Model of particle physics](http://www.particleadventure.org/standard-model.html), including the distinction between different types of neutrinos, the anti-particles for each of the above particles, and neutrons.  But this is enough to get reasonable cosmic ray physics!

We will consider __only the following interactions__ in our model:

* Proton downscattering: $p \rightarrow p + \pi + \pi$
* Pion decay: $\pi \rightarrow \mu + \nu$
* Muon decay: $\mu \rightarrow e + \nu$

The first process is due to collisions of the cosmic ray proton with the nuclei inside of air molecules in the Earth's atmosphere; this inelastic collision produces a pair of pions (it must be a pair due to charge conservation: the final state is really $p + \pi^+ + \pi^-$.)  The other two processes are decays of unstable particles, and would happen even in a vacuum.

Strictly speaking, there should be _two_ neutrinos in the decay of the muon - but we'll simplify by just considering one.  The pion can also decay as $\pi \rightarrow e + \nu$, but this is sufficiently rare that we can just ignore it.

We can break our understanding of all of these processes into three smaller questions:

1. What is the probability of a process happening, per unit time?
2. What does the process look like in the _rest frame_ of the initial particle?
3. Given the process in the rest frame, what does it look like in the _lab frame_ where we observe it?

### 1. Probability of decay/downscattering

As we saw in detail when dealing with Monte Carlo simulations, for a decay process with lifetime $\tau$, the probability that we observe a decay in time interval $dt$ is

$$
p(\tau, dt) = 1 - e^{-dt/\tau}.
$$

In our (laboratory) frame of reference, the three lifetimes we will need are:

$$
\tau_p = \exp \left( \frac{h}{7\ \textrm{km}} \right) \times \frac{1}{\beta} \times (1.14 \times 10^{-5}\ \textrm{s}) \\
\tau_{\pi} = \gamma \times (2.603 \times 10^{-8}\ \textrm{s}) \\
\tau_{\mu} = \gamma \times (2.197 \times 10^{-6}\ \textrm{s})
$$

where $\beta$ and $\gamma$ are the usual relativistic factors,

$$
\mathbf{\beta} = \frac{\mathbf{v}}{c}, \\
\gamma = \frac{1}{\sqrt{1-v^2/c^2}}.
$$

Note that $\mathbf{\beta} = (\beta_x, \beta_y)$ is a vector since we're working in two dimensions.  The variable $h$ is the height above the Earth's surface; the proton downscattering rate depends strongly on the density of the air, and thus on the height.

The first process, $p \rightarrow p + \pi + \pi$, arises from scattering off of air molecules; as a result, the time between scatters is inversely proportional to the speed of the proton, $\beta$.  Our model assumes a simple model for the density for air (see appendix C).  The other two processes are decays; in the rest frame of a $\pi$ or $\mu$ the lifetime is constant, but when they move at high speeds their apparent lifetime is extended due to __time dilation__.

One more complication: the proton scattering requires the proton to exceed a minimum "__threshold energy__" for the interaction to take place at all.  (A lot of energy is used up to convert to the mass of the two pions that are created.)  Assuming the nucleus being recoiled from is very heavy, the threshold for this interaction is

$$
E_p \geq (m_p + 2m_\pi) c^2 \approx 1218\ \rm{MeV}.
$$

__For a proton with less energy than the threshold, the downscattering _cannot occur_;__ in our model we will just assume that such a proton doesn't interact anymore.


### 2. Finding the decay products in the rest frame

Let's start with decay (processes 2 and 3).  In the rest frame of the mother particle, the system before and after decay looks like this:

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/final_projects/proj1_cosmic/proj1_decay.png" width=400px />

Conservation of momentum works for __any value__ of the angle $\theta$; quantum mechanics predicts the probability of any particular $\theta$.  We will assume the decay is __isotropic__, which means that all $\theta$'s are equally likely.  Once $\theta$ is chosen, conservation of momentum tells us the rest - a quick calculation (see appendix) gives

$$
|\mathbf{p}| = \frac{c(M_X^2 - M_Y^2)}{2M_X}
$$

where $M_X$ is the mass of the mother particle, and $M_Y$ is the mass of the heavy daughter particle.  (The other daughter particle is always a neutrino, so we take $M_Z = 0$.)


### 2. (continued) Downscattering in the rest frame

Now the other process.  To avoid running into problems with energy conservation, we need to include the heavy nitrogen-14 nucleus in the scattering.  We take it to be at rest in the lab frame, which means it will be incoming at high speed when we switch to the proton rest frame:

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/final_projects/proj1_cosmic/proj1_downscatter.png" width=500px />


In this case, it turns out that on top of the angles, the _momentum of each pion_ is random and determined by quantum effects.  (It has to be, since momentum conservation alone doesn't tell us what the pion momenta should be!)  A simple approximate model for the pion momentum distribution is

$$
p(E_\pi) = b e^{-b(E_\pi - m_\pi c^2)}
$$

where the coefficient $b = 1/(500\ \rm{MeV})$, and $E_\pi = \sqrt{|\mathbf{p}_\pi|^2 c^2 + m_\pi^2 c^4} \geq m_\pi c^2$.  We can draw from this distribution by drawing $y$ from the uniform distribution over $[0,1]$, and then computing

$$
E_\pi = m_\pi c^2 - \frac{1}{b} \log (1-y),
$$

and then the pion momentum is

$$
p_\pi = \sqrt{E_\pi^2 - m_\pi^2 c^4} / c.
$$

We again assume _isotropic_ pion creation, which means we pick random angles $\theta_1, \theta_2$ for both pion momentum vectors.  

The creation of two pions from nothing requires a large amount of energy, which (in this frame) comes from the kinetic energy of the nitrogen-14 nucleus.  Assuming the extra recoil energy of the proton is small, we have the relation

$$
E_{N}' \approx E_N - E_{\pi,1} - E_{\pi,2}
$$

where $E_N$ and $E_{N}'$ are the energy of the nitrogen-14 nucleus before and after the collision, respectively.  We can use this to compute the magnitude of the nitrogen-14 momentum after collision as

$$
|\mathbf{p}_N'| = \frac{1}{c} \sqrt{E_{N}'{}^2 - m_N^2 c^4}.
$$


Making one more assumption, that the _direction_ of $\vec{p}_N'$ is the same as the direction of the incident momentum $\mathbf{p}_N$ (which is true in the limit that the nitrogen-14 mass is very heavy), we can compute the components of $\mathbf{p}_N'$ by rescaling them using the ratio of the magnitudes $|\mathbf{p}_N'| / |\mathbf{p}_N|$.  This means the __change in momentum__ of the nitrogen-14 nucleus is:

$$
\Delta \mathbf{p}_{N'} = \mathbf{p}_N \left(1 - \frac{|p_{N'}|}{|p_N|} \right)
$$

Finally, we can determine the proton's momentum by adding up initial and final states:

$$
\mathbf{p}_p = \Delta \mathbf{p}_{N'} - \mathbf{p}_{\pi,1} - \mathbf{p}_{\pi,2}.
$$

Just to review, since that was a little complicated, the downscattering algorithm is:

1. Choose random momentum values for the pions, along with a random direction for each pion momentum.
2. Calculate the energy loss of the nitrogen-14 nucleus.
3. From the new energy and the old momentum, calculate the change in momentum vector of the nitrogen-14 nucleus.
4. Calculate the proton's final momentum using conservation of momentum.


### 3. Boosting back to the lab frame

Once we have momentum vectors in the rest frame of an interaction, we have to apply a __Lorentz transformation__ (a "boost") to go back to the lab frame.  In two dimensions, the lab components of the momentum vector are given by

$$
p_{x,\rm{lab}} = p_x - \gamma \frac{E}{c} \beta_x + (\gamma - 1) \beta_x \frac{\beta_x p_x + \beta_y p_y}{\beta^2} \\
p_{y,\rm{lab}} = p_y - \gamma \frac{E}{c} \beta_y + (\gamma - 1) \beta_y \frac{\beta_x p_x + \beta_y p_y}{\beta^2}
$$

now defining two _directional_ velocity factors $(\beta_x, \beta_y) = (v_x / c, v_y / c)$, so that $\beta = \sqrt{\beta_x^2 + \beta_y^2}$.  The right-hand side requires $p_x, p_y$, and relativistic energy $E$ in the rest frame of the decaying particle, and then $\beta_x, \beta_y, \gamma$ are taken from the lab-frame velocity of the decaying particle.

Finally, to get the speed from the momentum, starting from the relativistic formula for a massive particle

$$
\mathbf{p} = \gamma m \mathbf{v} = \frac{m\mathbf{v}}{\sqrt{1 - v^2/c^2}},
$$

we can show (see appendix A) that

$$
\mathbf{v} = \frac{\mathbf{p}/m}{\sqrt{1 + (p/mc)^2}}
$$

for a massive particle.  (If we have a _massless_ particle like a neutrino, then its speed is always just $c$, pointing in the same direction as $\mathbf{p}$.  Your code should treat this as a special case!)

# Computational Strategy and Algorithms

There are a lot of parts to keep track of in a cosmic ray shower!  The good news is that once we implement each small piece of physics above, we can use the magic of Monte Carlo simulation to chain them all together easily.  Aside from the Monte Carlo, all we need are some basic kinematics (force-free motion) to keep track of particle positions.  Here is the overall algorithm we want to use:

1. Keep track of the following info for all particles: __identity__ (proton, muon, ...), current __position__, and current __velocity__.
2. At each timestep $dt$, we use Monte Carlo to test for random decay/interaction, and split into multiple particles if such an event happens.  (For the proton scattering only, we check to see if $E_p$ is above the threshold first.)
3. Otherwise, we advance the positions of all particles as $(\Delta x, \Delta y) = (v_x dt, v_y dt)$.

All of the complications happen in that second step.  Whenever one of the three interaction processes from above happens, we find the outcome with the following sub-algorithm:

1. Choose a random direction for one of the decay products (for the proton, choose two random directions and two random momenta for each of the two pions.)
2. The momentum vectors of all decay products are then determined using conservation of energy and momentum.
3. Using the known velocity of the decaying particle, apply a Lorentz boost to find the momentum vectors of all decay products in the lab frame.
4. Convert from momentum to velocity, and update the list of particles.


# Application Programming Interface (API) specification

__You must implement all of the below functions, according to the specifications given.__  

Two commonly-used tuple structures in our API are the __particle__ and the __decay product__:

```
my_particle = (name, p_x, p_y, traj_x, traj_y)
my_product = (name, p_x, p_y)
```

Here `name` is a string, one of: `'proton'`, `'pion'`, `'muon'`, `'electron'`, or `'neutrino'`.  `p_x` and `p_y` are the momentum components of a particle in some reference frame.  `traj_x` and `traj_y` are arrays of distances, corresponding to the history of a particle's position.

(Why not just use "particle" tuples for everything?  A design decision to avoid potential confusion.  "Product" tuples will be created _outside_ the lab frame, and they won't have any position assigned to them until we do some work.)

You can use __any system of units for this project__, and things will work _as long as you're self-consistent!_  I recommend "particle physics units", in which the speed of light is set to $c = 1$: this puts energy, momentum, and mass all in common units of energy.  My code uses MeV for energy and s for time, which automatically requires that all distances are given in units of light-seconds (since $c$ is 1.)  The corresponding conversion factor is

$$
1\ {\rm ls} = 2.997 \times 10^5\ {\rm km} 
$$

### Special relativity:
* `lorentz_beta_gamma(beta_x, beta_y)`: Computes and returns the dimensionless Lorentz factors `(beta, gamma)` from the components of velocity.
* `boost_momentum(m, px, py, beta_x, beta_y)`: Computes and returns `(px_boost, py_boost)`, the components of momentum in the new frame of reference after a boost by $(\beta_x, \beta_y)$.
* `momentum_to_velocity(m, px, py)`: Returns the dimensionless velocity (`beta_x`, `beta_y`) corresponding to the given momentum and mass.
* `velocity_to_momentum(m, beta_x, beta_y)`: Returns the momentum vector components (`p_x`, `p_y`) corresponding to the given dimensionless velocity and mass.  Not used in the main loop, but useful for initializing particles with a given speed.

### Particle interactions:
* `check_for_interaction(particle, dt, h)`: Checks to see if a particle will interact within time `dt` at height `h` above the Earth's surface (so $h=0$ is at the surface, which we take to be average sea level.)  Returns `True` if a decay or downscatter occurs, and `False` otherwise.
* `particle_decay(name)`: Simulates the decay of a pion or muon in its rest frame.  Returns a tuple `(product_Y, product_Z)` containing _decay product_ tuples for each decay product.
* `proton_downscatter(beta_x , beta_y)`: Simulates the downscattering of a proton in its rest frame.  The inputs are the current components of relativistic velocity of the proton, which are used to determine the momentum of the recoiling nitrogen-14 nucleus in the proton rest frame.  Returns a tuple `(product_pi_1, product_pi_2, product_p)` containing _decay product_ tuples for each downscattering product.

### Other functions:
* `boost_product_to_lab(product, beta_x, beta_y)`: Given a decay product tuple `(name, px, py)` (see above) and the two components of velocity `(beta_x, beta_y)` of the parent particle, returns the tuple `(name, p_x_lab, p_y_lab)` giving the product's momentum in the lab frame, i.e. after boosting the momentum by $(-\beta_x, -\beta_y)$ (_note the minus signs since we're boosting __to the lab frame__.)_
* `particle_mass(name)`: Given a particle name, return its mass (in whatever units you are using.)


### Testing:
* `run_API_tests()`: A custom function that should use assertions to test the other functions implemented in the API.  If all tests are passed, the function should simply return the `None` object.  You should implement __at least six (6) tests__ inside this function on your own.

## Main loop

The below code implements the "main loop", running the Monte Carlo given an initial state, timestep, and number of timesteps to evolve.  Once you have implemented the API above, add this code to your `cosmic.py` module, then call it to run the Monte Carlo simulation!

In [ ]:
def run_cosmic_MC(particles, dt, Nsteps, attenuation=True):
    """
    Main loop for cosmic ray Monte Carlo project.

    Arguments:
    =====
    * particles: a list of any length, containing "particle" tuples.
        A particle tuple has the form:
            (name, p_x, p_y, traj_x, traj_y)
        where p_x and p_y are momentum vector components, and
        traj_x and traj_y are NumPy arrays of distance.

    * dt: time between Monte Carlo steps, in seconds.
    * Nsteps: Number of steps to run the Monte Carlo before returning.
    * attenuation: if True, protons and muons with too little energy are pruned
      from the simulation.  (This is a crude simulation of the real-world effect
      that charged particles moving through the air will lose energy.)

    Returns:
    =====
    A list of particle tuples.

    Example usage:
    =====

    >> init_p_x, init_p_y = velocity_to_momentum(particle_mass('muon'), 0.85, -0.24) # beta ~ 0.8, relativistic
    >> init_particles = [ ('muon', init_p_x, init_p_y, np.array([0]), np.array([1e-4]) ) ]  # ~ 30 km height in light-sec
    >> particles = run_cosmic_MC(init_particles, 1e-5, 100)

    The 'particles' variable after running should contain three particle tuples: the
    initial muon, an electron, and a neutrino.  (Even though the muon decays,
    we keep it in the final particle list for its trajectory.)

    """

    E_threshold = 2000  # MeV - change this constant if your unit system is different!!

    stopped_particles = []
    for step_i in range(Nsteps):
        updated_particles = []

        for particle in particles:
            # Unpack particle tuple
            (name, p_x, p_y, traj_x, traj_y) = particle
            (beta_x, beta_y) = momentum_to_velocity(particle_mass(name), p_x, p_y)

            # Check for interaction
            does_interact = check_for_interaction(particle, dt, traj_y[-1])

            if does_interact:
                if name == "proton":
                    decay_products = proton_downscatter(beta_x, beta_y)
                else:
                    stopped_particles.append(particle)
                    decay_products = particle_decay(name)

                # Transform products back to lab frame
                for product in decay_products:
                    (product_name, product_p_x, product_p_y) = boost_product_to_lab(
                        product, beta_x, beta_y
                    )

                    ## Clean up - attentuation of particles with too little energy
                    if attenuation:
                        E_product = np.sqrt(
                            product_p_x**2
                            + product_p_y**2
                            + particle_mass(product_name) ** 2
                        )
                        if E_product < E_threshold:
                            if product_name == "proton":
                                # Avoid losing proton tracks, since downscatter doesn't add stopped particles
                                stopped_particles.append(particle)
                            if product_name != "neutrino":
                                # Skip decay product, don't append to updated_particles
                                continue

                    # If this was a proton scatter, then the "new" proton is
                    # the same as the original, so keep track of its trajectory!
                    if name == "proton" and product_name == "proton":
                        product_traj_x = traj_x
                        product_traj_y = traj_y
                    else:
                        product_traj_x = np.array([traj_x[-1]])
                        product_traj_y = np.array([traj_y[-1]])

                    # Make new particle tuple and append
                    product_particle = (
                        product_name,
                        product_p_x,
                        product_p_y,
                        product_traj_x,
                        product_traj_y,
                    )
                    updated_particles.append(product_particle)
            else:
                # Doesn't interact, so compute motion
                traj_x = np.append(traj_x, traj_x[-1] + beta_x * dt)
                traj_y = np.append(traj_y, traj_y[-1] + beta_y * dt)

                updated_particles.append((name, p_x, p_y, traj_x, traj_y))

        # Run next timestep
        particles = updated_particles

    # Add stopped particles back to list and return
    particles.extend(stopped_particles)
    return particles

---
---
---

The below appendices are included for your information, but you don't need to understand them in gory detail for the project - just use the resulting formulas as given above.

## Appendix A: Kinematics for decay/scattering

Here I show the full derivations for the equations governing decay and scattering in our simplified model.  Let's start with the simpler case of $X \rightarrow Y + \nu$ decay for processes 2 and 3 above.  With only two particles in the final state, momentum conservation requires them to be back-to-back.  Once the angle $\theta$ is fixed (by randomly choosing it), the only quantity left to determine is the magnitude $p$ of the momentum vectors.

For a relativistic system, the expression for energy is $E = \sqrt{p^2c^2 + m^2c^4}$.  The mass of the $\nu$ is zero, so applying conservation of energy to the initial and final states, we find

$$
E_X = E_Y + E_Z \\
m_X c^2 = \sqrt{p^2 c^2 + m_Y^2 c^4} + pc
$$

which we can solve to find

$$
p = \frac{c(m_X^2 - m_Y^2)}{2m_X}.
$$


Now, proton downscattering.  The details for these were laid out above, but an important assumption we made is that the nucleus scattered off of in Earth's atmosphere is _not_ traveling relativistically, so our incoming proton must have enough energy __in the lab frame__ to produce a pair of pions.  This minimum is called the __threshold energy__.  

For $p \rightarrow p + \pi + \pi$, the minimum possible energy occurs if all three products are at rest, which yields

$$
E_p \geq (m_p + 2m_\pi) c^2 \approx 1218\ \rm{MeV}.
$$

(If we work this out carefully including the air nucleus we're scattering off of, we find the same result in the limit that the nucleus is very heavy; nitrogen-14 is heavy enough that we don't have to modify this estimate for our purposes.)

## Appendix B: Lorentz boosts in two spatial dimensions 

Most intro textbooks that discuss special relativity just show the effects of a Lorentz boost only along one of the coordinate axes.  For this project, we need the more complicated expression for a coordinate change in an arbitrary direction.  The mixing between coordinates can be described by a __boost matrix__ of the form

$$
\left(\begin{array}{c}ct' \\ x' \\ y'\end{array} \right) = 
\left( \begin{array}{ccc} 
    \gamma & -\gamma \beta_x & -\gamma \beta_y \\ 
    -\gamma \beta_x & 1 + \frac{\gamma-1}{|\beta|^2} \beta_x^2 & \frac{\gamma-1}{|\beta|^2} \beta_x \beta_y \\
    -\gamma \beta_y & \frac{\gamma-1}{|\beta|^2} \beta_x \beta_y & 1 + \frac{\gamma-1}{|\beta|^2} \beta_y^2
\end{array} \right) \cdot \left( \begin{array}{c} ct \\ x \\ y \end{array} \right)
$$

Textbook version: for boost by speed $\beta_x = v_x/c$ along the x-axis, we have

$$
t' = \gamma(t - \beta_x x) \\
x' = \gamma(x - \beta_x ct) \\
y' = y
$$

In general, for a boost by $\boldsymbol{\beta} = (\beta_x, \beta_y)$ in a general direction, we can write a vector version of these equations:

$$
t' = \gamma(t - \boldsymbol{\beta} \cdot \mathbf{x}) \\
\mathbf{x}' = \mathbf{x} - \gamma ct \boldsymbol{\beta} + \frac{\gamma - 1}{|\boldsymbol{\beta}|^2} (\boldsymbol{\beta} \cdot \mathbf{x}) \boldsymbol{\beta}
$$

You can verify that this matches the "boost matrix" above.  As a simple check, notice that this reduces to the textbook version if $\boldsymbol{\beta} = (\beta_x, 0)$.  For our purposes, we can either use the vector version, or we can write out the components to find:



$$
t' = \gamma(t - \beta_x x - \beta_y y) \\
x' = x - \gamma c t \beta_x + (\gamma - 1) \beta_x \frac{\beta_x x + \beta_y y}{\beta^2} \\
y' = y - \gamma c t \beta_y + (\gamma - 1) \beta_y \frac{\beta_x x + \beta_y y}{\beta^2}
$$

Exactly the same equations hold for the energy-momentum vectors: just replace $(t, \mathbf{x})$ with $(E, \mathbf{p})$.

$$
\mathbf{p} = \gamma m \mathbf{v} = \frac{m\mathbf{v}}{\sqrt{1 - v^2/c^2}},
$$

which we can invert first by noticing that

$$
p^2 = \gamma^2 m^2 v^2 = \gamma^2 m^2 c^2 (1 - \frac{1}{\gamma^2}) \\
\frac{p^2}{m^2 c^2} = \gamma^2 - 1 \\
\gamma = \sqrt{1 + (p/mc)^2}
$$

to get

$$
\mathbf{v} = \frac{\mathbf{p}/m}{\sqrt{1 + (p/mc)^2}}
$$

In [ ]:
# Calculation for appendix C below
import numpy as np

1.23 * np.exp(-15 / 7)
NA = 6.022e23
1.23 * NA / 28.9644e-3

## Appendix C: Finding the rate for proton downscattering

As discussed, the process we write as $p \rightarrow p + \pi + \pi$ is really a two-to-four scattering process, $p + X \rightarrow p + X + \pi + \pi$, where $X$ is a nucleus inside of an air molecule.  The rate at which this will occur is described by a _cross section_, $\sigma \approx 114$ millibarn (very roughly, based on experiments on nitrogen nuclei and then corrected for the fact that we always scatter with enough energy to make pions.)  A millibarn is equal to $10^{-31}$ square meters.  The inverse scattering rate (average time to scatter) is related to the cross section as

$$
\tau = 1/(\sigma n v)
$$

where $v$ is the speed of the proton and $n$ is the number density of air.  An important effect is that the density of air decreases rapidly further up from the Earth's surface.  We will adopt [a simple isothermal model](https://www.spaceacademy.net.au/watch/debris/atmosmod.htm), in which the mass density of the atmosphere at height $h$ is equal to

$$
\rho(h) \approx 1.23\ {\rm kg/m}^3 \exp \left(-\frac{h}{7\ {\rm km}} \right)
$$

or translating to number density using $n = (N_A / M) \rho$,
$$
n(h) \approx 2.56 \times 10^{25}\ {\rm m}^{-3} \exp \left(-\frac{h}{7\ {\rm km}} \right).
$$

Rewriting $v$ as $\beta c$, we find that

$$
\tau \approx \exp \left( \frac{h}{7\ {\rm km}} \right) \frac{1}{\beta} 1.14 \times 10^{-5}\ \rm{s}.
$$

Cosmic ray showers are most likely to be initiated in the stratosphere at an altitude of around 15 km.



## Appendix D: Drawing random pion energies

From quantum mechanics, an approximate model for the distribution of pions produced in proton downscattering gives the probability distribution

$$
p(E_\pi) = b e^{-b(E_\pi - m_\pi c^2)}.
$$

To draw randomly from this distribution, we use the [inverse transform sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling) method.  First, we integrate to find the cumulative distribution function:

$$
F(E_\pi) = 1 - e^{-b(E_\pi - m_\pi c^2)}
$$

Next, we set $y = F(E_\pi)$ and invert to find $E_\pi$ as a function of $y$:

$$
y = 1 - e^{-b(E_\pi - m_\pi c^2)} \\
e^{-b(E_\pi - m_\pi c^2)} = 1 - y \\
-b(E_\pi - m_\pi c^2) = \log (1-y) \\
E_\pi = m_\pi c^2 - \frac{1}{b} \log (1-y)
$$

Now drawing $y$ from the uniform distribution over $[0,1]$ gives the $E_\pi$ distributed as we want.  (Read the link above to see why this trick works, but the one-sentence explanation is that the CDF $F(E_\pi)$ is also distributed over $[0,1]$ by definition.)